In [ ]:
#This is the notebook for tools_project

In [1]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [3]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat now ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat now ? ramen
Where are you ? Boston, MA


'https://www.yelp.com/search?find_desc=ramen&find_loc=Boston, MA'

In [4]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [5]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [6]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [7]:
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="bOfM3D6YxYDvMy3smHxpQQ" href="/adredir?ad_business_id=UgAVU0n3ysKBVoNWp4OHcg&amp;campaign_id=5qgIavK7B69wOJWJ5kUz6Q&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fshabu-and-mein-cambridge-3&amp;request_id=aefc6171b0da04ed&amp;signature=4c0b8abe53c117b74a7814898ba57ac1b18188455b8e868cff0b2767b3bc1669&amp;slot=0"><span>Shabu &amp; Mein</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="G_curU_foguyxUTX9lEuPQ" href="/biz/santouka-back-bay-boston-2?osq=ramen"><span>Santouka Back Bay</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="aAJQ25y_aZ5WMesyjtGcgg" href="/biz/wens-noodle-and-ramen-%E8%BF%87%E6%A1%A5%E7%B1%B3%E7%BA%BF-boston?osq=ramen"><span>Wens noodle and Ramen 过桥米线</span></a>,
 <a class="biz-name js-analytics-click" data-anal

In [8]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this perticular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [9]:
print(name,link)

Santouka Back Bay https://www.yelp.com/biz/santouka-back-bay-boston-2?osq=ramen


In [10]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [11]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [12]:
json_map

'{"serviceAreas": [], "moMapPossible": true, "scrollwheelZoom": false, "zoomControlPosition": "top_right", "minZoomlevel": null, "isFullBleed": false, "maxZoomlevel": null, "zoom": 15, "library": "google", "fitToGeobox": false, "hoods": [], "adPinColor": null, "markers": [{"location": null, "key": "directions_marker", "icon": {"name": "directions", "anchorOffset": [12, 32], "activeOrigin": [0, 0], "scaledSize": [24, 32], "regularUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/79f63ebc20db/assets/img/directions@2x.png", "size": [24, 32], "activeUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/7249ab345ac8/assets/img/directions_highlighted@2x.png", "regularOrigin": [0, 0]}}, {"resourceType": "business", "url": "/biz/santouka-back-bay-boston-2", "resourceId": "inAEY3i1hYR4BlJfPgKFPw", "shouldOpenInNewTab": false, "location": {"latitude": 42.3484762870602, "longitude": -71.0854271426797}, "key": "starred_business", "hovercardId": "G_curU_foguyxUTX9lEuPQ", "icon":

In [13]:
print(latitude,longitude)

42.3484762870602 -71.0854271426797


In [14]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [15]:
category

'Ramen'

In [16]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [17]:
print(price_range)

$$


In [18]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [19]:
print(rating_value,review_count)

4.0 398


In [20]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [21]:
print(phone_number)

(857) 310-5194


In [22]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [23]:
print(open_)

None


In [ ]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

In [ ]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

In [24]:
#want to put all infomations we want to get from one business in to one funtion
#input: url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
    dict_map = json.loads(json_map)
    latitude = dict_map["markers"][1]["location"]["latitude"]
    longitude = dict_map["markers"][1]["location"]["longitude"]

    category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    ratings = results_business_page.find("div", itemprop = 'aggregateRating')
    rating_value = ratings.find("meta").get('content')
    review_count = ratings.find("span").get_text()
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = None
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
        
    return(addr,latitude,longitude,category,price_range,rating_value,review_count,phone_number,open_,web)
    

In [25]:
#test the function on cafe mogador
restaurant_url = "https://www.yelp.com/biz/cafe-mogador-brooklyn?osq=Restaurants"
get_info(restaurant_url)

('133 Wythe AveBrooklyn, NY 11211',
 40.719747,
 -73.959993,
 'Moroccan',
 '$$',
 '4.5',
 '1004',
 '(718) 486-9222',
 'Open now',
 'cafemogador.com/williamsburg')

In [45]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants

def linklist():
    link_list = []
    for i in range(1,len(businesses)):
        business = businesses[i]
        link = 'https://www.yelp.com' + business.get('href')
        link_list.append(link)
    return link_list

In [47]:
linklist()

['https://www.yelp.com/biz/santouka-back-bay-boston-2?osq=ramen',
 'https://www.yelp.com/biz/wens-noodle-and-ramen-%E8%BF%87%E6%A1%A5%E7%B1%B3%E7%BA%BF-boston?osq=ramen',
 'https://www.yelp.com/biz/neponset-cafe-boston-2?osq=ramen',
 'https://www.yelp.com/biz/amateras-ramen-boston?osq=ramen',
 'https://www.yelp.com/biz/santouka-harvard-square-cambridge-3?osq=ramen',
 'https://www.yelp.com/biz/soup-shack-boston?osq=ramen',
 'https://www.yelp.com/biz/ganko-ittetsu-ramen-brookline?osq=ramen',
 'https://www.yelp.com/biz/oisa-ramen-boston-2?osq=ramen',
 'https://www.yelp.com/biz/yume-wo-katare-cambridge-2?osq=ramen',
 'https://www.yelp.com/biz/tsurumen-davis-somerville?osq=ramen',
 'https://www.yelp.com/biz/ruckus-boston-2?osq=ramen',
 'https://www.yelp.com/biz/shojo-boston-2?osq=ramen',
 'https://www.yelp.com/biz/shabu-and-mein-cambridge-3?osq=ramen',
 'https://www.yelp.com/biz/sakura-sunakku-boston?osq=ramen',
 'https://www.yelp.com/biz/pikaichi-medford-2?osq=ramen',
 'https://www.yelp.co

In [54]:
#Create a list of restaurant names

namelist=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    namelist.append(name)
        
namelist

['Santouka Back Bay',
 'Wens noodle and Ramen 过桥米线',
 'Neponset Cafe',
 'アマテラス ラーメン',
 'Santouka Harvard Square',
 'Soup Shack',
 'Ganko Ittetsu Ramen',
 'Oisa Ramen',
 'Yume Wo Katare',
 'Tsurumen Davis',
 'Ruckus',
 'Shojo',
 'Shabu & Mein',
 'Sakura Sunakku',
 'Pikaichi',
 'Ittoku',
 'Chiharu',
 'Hakata Ramen & Sushi',
 'Shin Hakata Ramen',
 'Yume Ga Arukara',
 'Sushi Kappo',
 'Noodle Lab',
 'Snappy Kitchen',
 'Little Big Diner',
 'One Ramen & Sushi',
 'PABU Boston',
 'Tora Japanese Restaurant',
 'Chick Chick Boom',
 'Totto Ramen',
 'Sapporo Ramen']